In [43]:
import pandas as pd
import xlrd
import os
from collections import defaultdict

In [44]:
#from PyPDF2 import PdfFileReader,PdfFileWriter

In [45]:
ls ./data

2019_12/          2020_04/          2020_05的副本/    季度汇总/
2020_01/          2020_04的副本/    2020_06/
2020_03/          2020_05/          2020_07/


In [46]:
class Bill:
    def __init__(self, year, mon):
        self.year = year
        self.mon = mon
        self.sell_file = self.get_sell_file()
        content = xlrd.open_workbook(filename=self.sell_file,encoding_override='gbk')
        self.sell_df = pd.read_excel(content)
        self.commission_file = self.get_commission_file()
        content = xlrd.open_workbook(filename=self.commission_file,encoding_override='gbk')
        self.commission_df = pd.read_excel(content)
        #self.sell_df = pd.read_excel(self.sell_file,encoding='gbk')
    def gen_path(self, filename):
        return  './data/{0}_{1:02d}/{2}'.format(self.year, self.mon,filename)
    def get_sell_file(self):
        return self.gen_path('销售单据表.xls')
    def get_commission_file(self):
        return self.gen_path('提成明细表.xls')
    def get_total_income(self):
        #总收入
        self.total_income = 0
        #充值总收入
        self.total_recharge = 0
        #充次总收入
        self.total_time_recharge = 0
        for index, row in self.sell_df.iterrows():
            income_sf = row['实付']
            income_czk = row['储值卡']
            self.total_income += row['实付'] - row['储值卡']
            
            if row['单据类型'] == '购买次数':
                self.total_time_recharge += row['实付']
                
            if row['单据类型'] == '购买次数' or row['单据类型'] == '充值收费' or row['单据类型']=='开户收费':
                self.total_recharge += row['实付'] - row['储值卡']
               
        print('总收入:',self.total_income)
        print('充值总收入:',self.total_recharge)
        print('充次总收入:',self.total_time_recharge) 
    def get_true_income(self):
        # 实际项目总收入（会员+非会员）
        self.true_income = 0
        true_income_types = ['消费收银','快速消费']
        #会员实际项目总收入
        self.member_true_income = 0
        for index, row in self.sell_df.iterrows():
            if row['单据类型'] in true_income_types:
                if not pd.isnull(row['会员类型']):
                    self.member_true_income += row['实付']
                self.true_income += row['实付']
        print(self.member_true_income)
        print(self.true_income)
        
        #实际扣次项目
        self.true_time_dic = defaultdict(int)
        for index, row in self.commission_df.iterrows():
            if row['提成类型'] == '扣次消费':
                self.true_time_dic[row['名称']] += 1
        print(self.true_time_dic)
    def get_employee_commission(self):
        tmp_commission = self.commission_df.copy()
        self.commision_detail = {}
        selected_df = tmp_commission[['时间','员工','名称','支付','提成','提成类型','姓名']]
        selected_df['时间'] = selected_df['时间'].apply(lambda x:'{0}-{1}'.format(x.month, x.day))
        for employee, df in selected_df.groupby(['员工']):
            print(employee)
            print(df)
            df = df[['时间','员工','名称','提成类型','姓名','支付','提成']]
            print(df['提成'].sum())
            self.commision_detail[employee +"_提成"] = df['提成'].sum()
            df.to_csv(self.gen_path('{0}.csv'.format(employee)),index=False,encoding='gbk')
            df.to_excel(self.gen_path('{0}.xls'.format(employee)),index=False,encoding='gbk')
        self.commision_detail['总提成'] = tmp_commission['提成'].sum()
        print(self.commision_detail)
        
    def get_recharge_detail(self):
        tmp_df = self.sell_df.copy()
        recharge_df = tmp_df[tmp_df['单据类型'].isin(set(['购买次数','充值收费','开户收费']))]
        recharge_df = recharge_df[['单据类型','时间','实付','储值卡','会员姓名','会员卡号']]
        total_recharge = recharge_df['实付'].sum()-recharge_df['储值卡'].sum()
        des_sum = '累计充值：{0},各个会员充值情况如下\n'.format(total_recharge)
        des_sum += '==============================\n'
        for name, df in recharge_df.groupby(['会员卡号']):
            s = 0
            des = '{0}({1:d}):累计充值{2}元\n'.format(df.iloc[0]['会员姓名'],int(name),int(df['实付'].sum()-df['储值卡'].sum()))
            des_sum += des
            #print('{0}:{1}'.format(name,df['实付'].sum()-df['储值卡'].sum()))
            total_recharge += df['实付'].sum()-df['储值卡'].sum()
            s += df['实付'].sum()-df['储值卡'].sum()
            des = '明细如下:\n'
            df = df.sort_values(by=['单据类型'],ascending=False)

            for index, row in df.iterrows():
                if row['单据类型'] == '购买次数' and row['实付']>0:
                    des = des + '购买次数:' + str(int((row['实付']))) + ',时间:' + (str(row['时间'].month) + '-' + str(row['时间'].day))+'\n'
                    s -= row['实付']
                if row['单据类型'] == '充值收费' and s > 0:
                    des = des + '充值收费:' + str((int(row['实付']))) + ',时间:' + (str(row['时间'].month) + '-' + str(row['时间'].day))+'\n'
                    s -= row['实付']
                if row['单据类型'] == '开户收费' and s > 0:
                    des = des + '开户收费:' + str((int(row['实付']))) + ',时间:' + (str(row['时间'].month) + '-' + str(row['时间'].day))+'\n'
                    s -= row['实付']
            #print(des)
            des_sum += des
            des_sum += '==============================\n'
            f = open(self.gen_path('会员充值明细_{0}_{1}.txt'.format(self.year,self.mon)),'w')
            f.write(des_sum)
            f.close()
        return des_sum
    def get_nomember_detail(self):
        tmp_df = self.sell_df.copy()
        tmp_df['not_member'] = tmp_df['会员卡号'].apply(lambda x:pd.isnull(x))
        nomember_detail_df = tmp_df[tmp_df['not_member']]
        nomember_detail_df = nomember_detail_df[['单据类型','实付','时间']]
        #print(nomember_detail_df)
        des_not_member = ''
        des_not_member += '散客累计消费:{0}\n明细如下：\n'.format(nomember_detail_df['实付'].sum())
        for index, row in nomember_detail_df.iterrows():
            des_not_member += '{0}:'.format(row['单据类型']) + str((int(row['实付']))) + ',时间:' + (str(row['时间'].month) + '-' + str(row['时间'].day))+'\n'
        f = open(self.gen_path('散客消费明细_{0}_{1}.txt'.format(self.year,self.mon)),'w')
        f.write(des_not_member)
        f.close()
        return nomember_detail_df['实付'].sum()

In [47]:
ls ./data/2020_04/

孙妮.csv                       邵红梅.xls
黄晴.csv                       提成明细表.xls
孙妮.xls                       销售单据表.xls
黄晴.xls                       会员充值明细_2020_4.txt
邵红梅.csv                     散客消费明细_2020_4.txt


In [48]:
bill = Bill(2020, 7)

In [49]:
bill.get_sell_file()

'./data/2020_07/销售单据表.xls'

In [50]:
bill.get_true_income()

4756.599999999999
6974.599999999999
defaultdict(<class 'int'>, {'美睫': 1, '暖宫艾灸': 6, '精油开背': 5, '皮肤清洁护理': 1})


In [51]:
bill.sell_file

'./data/2020_07/销售单据表.xls'

In [52]:
bill.get_total_income()

总收入: 8178.0
充值总收入: 5960.0
充次总收入: 4160.0


In [53]:
bill.commission_file

'./data/2020_07/提成明细表.xls'

In [54]:
bill.get_employee_commission()

孙妮
      时间  员工    名称      支付      提成  提成类型   姓名
0    7-4  孙妮   NaN    99.0   39.60  快速消费  小米粥
1    7-4  孙妮    美睫   194.0  107.00  扣次消费  郭燕萍
2    7-5  孙妮   NaN   100.0   40.00  快速消费  NaN
3    7-5  孙妮   NaN   393.0  157.20  快速消费   小郭
4    7-5  孙妮   NaN   200.0   80.00  快速消费  NaN
5    7-5  孙妮   NaN   388.0  155.20  快速消费   姣爷
6    7-5  孙妮   NaN    40.0   40.00  快速消费   姣爷
7    7-5  孙妮   NaN    50.0   30.00  快速消费  NaN
8    7-5  孙妮   NaN   109.0   43.60  快速消费  曹小姐
9    7-5  孙妮   NaN   194.0   77.60  快速消费   袁月
10   7-6  孙妮   NaN    96.6   38.64  快速消费   丁丁
11   7-6  孙妮   NaN   330.0  132.00  快速消费  NaN
12   7-6  孙妮   NaN   196.0   78.40  快速消费   黄燕
13   7-7  孙妮   NaN   294.0  117.60  快速消费  小倪姐
14   7-7  孙妮   NaN    99.0   39.60  快速消费   刘梅
15   7-7  孙妮   NaN   194.0   77.60  快速消费   张依
16   7-7  孙妮   NaN    30.0   20.00  快速消费   黄潇
17   7-7  孙妮   NaN   109.0   43.60  快速消费   黄潇
18  7-15  孙妮   NaN   100.0   40.00  快速消费  NaN
19  7-15  孙妮   NaN   110.0   25.00  快速消费  NaN
20  7-15  孙妮   NaN   500.0   25

/Users/mtdp/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [55]:
des_sum = bill.get_recharge_detail()

In [56]:
des_sum

'累计充值：5960.0,各个会员充值情况如下\n==============================\n刘红贵(60028):累计充值2860元\n明细如下:\n购买次数:880,时间:7-29\n购买次数:1980,时间:7-29\n==============================\n张玉华(60039):累计充值300元\n明细如下:\n充值收费:300,时间:7-20\n==============================\n凤蔚(60079):累计充值1800元\n明细如下:\n购买次数:1300,时间:7-15\n充值收费:500,时间:7-15\n==============================\n陈菊(60080):累计充值1000元\n明细如下:\n充值收费:1000,时间:7-30\n==============================\n'

In [57]:
des_not_member = bill.get_nomember_detail()

In [58]:
des_not_member

2218.0

In [59]:
bills  = []
bills.append(Bill(2020, 3))
bills.append(Bill(2020, 4))
bills.append(Bill(2020, 5))

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


In [60]:
rows = []
for bill in bills:
    row = {}
    bill.get_total_income()
    row['月份'] = bill.mon
    row['总收入'] = bill.total_income
    row['充值总收入'] = bill.total_recharge
    row['散客消费总收入'] = bill.get_nomember_detail()
    bill.get_employee_commission()
    print(bill.commision_detail)
    for k in  bill.commision_detail:
        row[k] = bill.commision_detail[k]
    rows.append(row)

总收入: 7392
充值总收入: 5999
充次总收入: 1999
孙妮
      时间  员工    名称   支付     提成  提成类型   姓名
11  3-20  孙妮   NaN   99   39.6  快速消费  小米粥
16  3-22  孙妮   NaN  230   92.0  快速消费  NaN
19  3-23  孙妮   NaN  388  155.2  快速消费  NaN
22  3-24  孙妮   NaN  263  105.2  快速消费   姣爷
24  3-24  孙妮   NaN  294  117.6  快速消费   小郭
27  3-26  孙妮   NaN  363  145.2  快速消费  小倪姐
28  3-26  孙妮  暖宫艾灸   88    8.0  扣次消费   潘潘
30  3-27  孙妮   NaN  194   77.6  快速消费   徐梅
31  3-27  孙妮   NaN  100   40.0  快速消费  NaN
32  3-27  孙妮   NaN  194   77.6  快速消费   潘潘
858.0
邵红梅
      时间   员工    名称    支付    提成  提成类型   姓名
4   3-16  邵红梅   NaN    69  27.6  快速消费   刘梅
5   3-19  邵红梅  精油开背    88   8.0  扣次消费  沈慧仙
20  3-23  邵红梅  暖宫艾灸    88   8.0  扣次消费   袁月
26  3-26  邵红梅   NaN  1000  50.0  充值收费   潘潘
93.6
黄晴
      时间  员工      名称    支付    提成  提成类型   姓名
0    3-3  黄晴    精油开背  1000  50.0  购买次数  小霸王
1    3-3  黄晴    精油开背    88  30.0  扣次消费  小霸王
2   3-16  黄晴  皮肤清洁护理   132  30.0  扣次消费  刘红贵
3   3-16  黄晴    精油开背    88  30.0  扣次消费  刘红贵
6   3-19  黄晴    精油开背    88  30.0  扣次消费  刘红贵
7   

/Users/mtdp/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/mtdp/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/mtdp/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

In [61]:
rows

[{'月份': 3,
  '总收入': 7392,
  '充值总收入': 5999,
  '散客消费总收入': 1393,
  '孙妮_提成': 858.0,
  '邵红梅_提成': 93.6,
  '黄晴_提成': 603.0,
  '总提成': 1554.6},
 {'月份': 4,
  '总收入': 4110,
  '充值总收入': 3500,
  '散客消费总收入': 610,
  '孙妮_提成': 1189.6999999999998,
  '邵红梅_提成': 101.2,
  '黄晴_提成': 76.0,
  '总提成': 1366.9},
 {'月份': 5,
  '总收入': 3358,
  '充值总收入': 2500,
  '散客消费总收入': 858,
  '孙妮_提成': 1633.7999999999997,
  '邵红梅_提成': 46.0,
  '黄晴_提成': 331.2,
  '总提成': 2010.9999999999998}]

In [62]:
result = pd.DataFrame(rows)

In [63]:
result.to_excel('季度收入汇总表.xlsx',index=False)